In [2]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.initializers import Constant
from keras.models import Sequential
from keras.utils import plot_model
from keras.layers import Dense, Embedding, Dropout, Flatten, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Embedding
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib.pyplot import figure
import seaborn as sn
import pandas as pd
import numpy as np
import string
import re

In [13]:
df=pd.read_csv('/kaggle/input/sspotufy-song-dataset/spotify_millsongdata.csv')

In [14]:
def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)  # Remove text in brackets
    text = re.sub(r'\n', ' ', text)      # Replace newlines with spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower().strip()          # Convert to lowercase and strip spaces
    return text

df['clean_lyrics'] = df['text'].apply(clean_text)

In [15]:
df


,artist,song,link,text,clean_lyrics
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",look at her face its a wonderful face \r and ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",take it easy with me please \r touch me gentl...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,ill never know why i had to go \r why i had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,making somebody happy is a question of give an...
...,...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...,irie days come on play \r let the angels fly ...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...,power to the workers \r more power \r power ...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...,all you need \r is something ill believe \r ...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...,northern star \r am i frightened \r where ca...


In [16]:
df=df.drop('link',axis=1)
df=df.drop('text',axis=1)


In [17]:
df

,artist,song,clean_lyrics
0,ABBA,Ahe's My Kind Of Girl,look at her face its a wonderful face \r and ...
1,ABBA,"Andante, Andante",take it easy with me please \r touch me gentl...
2,ABBA,As Good As New,ill never know why i had to go \r why i had t...
3,ABBA,Bang,making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...
...,...,...,...
57645,Ziggy Marley,Good Old Days,irie days come on play \r let the angels fly ...
57646,Ziggy Marley,Hand To Mouth,power to the workers \r more power \r power ...
57647,Zwan,Come With Me,all you need \r is something ill believe \r ...
57648,Zwan,Desire,northern star \r am i frightened \r where ca...


In [18]:
from transformers import pipeline

# Load the Hugging Face emotion classifier
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=3)

# Function to classify emotions
def classify_emotions(lyrics):
    if not lyrics or lyrics.strip() == "":
        return []  # Return empty list for empty lyrics
    try:
        results = emotion_classifier(lyrics[:512])  # Limit to the first 512 characters
        emotions = [res['label'] for res in results[0]]  # Access the first element and extract 'label'
        return emotions if emotions else ["neutral"]
    except Exception as e:
        print(f"Error processing lyrics: {e}")
        return []

# Apply the function to populate the 'emotions' column
df['emotions'] = df['clean_lyrics'].apply(classify_emotions)


Device set to use cuda:0


In [19]:
# Mapping emotions to moods
mood_mapping = {
    "joy": "happy",
    "love": "love",
    "anger": "angry",
    "sadness": "sad",
    "fear": "anxious",
    "surprise": "excited",
    "neutral": "neutral"
}

def map_moods(emotions):
    return [mood_mapping.get(emotion, "unknown") for emotion in emotions]

# Apply the mood mapping
df['moods'] = df['emotions'].apply(map_moods)


In [20]:
def recommend_songs(mood, df, top_n=10):
    
    recommended_songs = df[df['moods'].apply(lambda moods: mood in moods)]
    return recommended_songs[['artist', 'song']].head(top_n)

recommend_songs('happy', df, top_n=5)


,artist,song
0,ABBA,Ahe's My Kind Of Girl
1,ABBA,"Andante, Andante"
2,ABBA,As Good As New
3,ABBA,Bang
4,ABBA,Bang-A-Boomerang


In [21]:
def recommend_songs(mood, df, top_n=10):
    
    recommended_songs = df[df['moods'].apply(lambda moods: mood in moods)]
    return recommended_songs[['artist', 'song']].head(top_n)

recommend_songs('angry', df, top_n=5)


,artist,song
5,ABBA,Burning My Bridges
9,ABBA,Crying Over You
25,ABBA,Hamlet III
50,ABBA,Lay All Your Love On Me
56,ABBA,Mamma Mia


In [22]:
df

,artist,song,clean_lyrics,emotions,moods
0,ABBA,Ahe's My Kind Of Girl,look at her face its a wonderful face \r and ...,"[joy, surprise, sadness]","[happy, excited, sad]"
1,ABBA,"Andante, Andante",take it easy with me please \r touch me gentl...,"[joy, fear, sadness]","[happy, anxious, sad]"
2,ABBA,As Good As New,ill never know why i had to go \r why i had t...,"[sadness, surprise, joy]","[sad, excited, happy]"
3,ABBA,Bang,making somebody happy is a question of give an...,"[joy, surprise, sadness]","[happy, excited, sad]"
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...,"[joy, surprise, sadness]","[happy, excited, sad]"
...,...,...,...,...,...
57645,Ziggy Marley,Good Old Days,irie days come on play \r let the angels fly ...,"[sadness, joy, surprise]","[sad, happy, excited]"
57646,Ziggy Marley,Hand To Mouth,power to the workers \r more power \r power ...,"[sadness, surprise, anger]","[sad, excited, angry]"
57647,Zwan,Come With Me,all you need \r is something ill believe \r ...,"[sadness, surprise, joy]","[sad, excited, happy]"
57648,Zwan,Desire,northern star \r am i frightened \r where ca...,"[sadness, fear, joy]","[sad, anxious, happy]"


In [23]:
df['moods'].value_counts

<bound method IndexOpsMixin.value_counts of 0        [happy, excited, sad]
1        [happy, anxious, sad]
2        [sad, excited, happy]
3        [happy, excited, sad]
4        [happy, excited, sad]
                 ...          
57645    [sad, happy, excited]
57646    [sad, excited, angry]
57647    [sad, excited, happy]
57648    [sad, anxious, happy]
57649    [sad, excited, happy]
Name: moods, Length: 57650, dtype: object>

In [29]:
df


,artist,song,clean_lyrics,emotions,moods
0,ABBA,Ahe's My Kind Of Girl,look at her face its a wonderful face \r and ...,"[joy, surprise, sadness]","[happy, excited, sad]"
1,ABBA,"Andante, Andante",take it easy with me please \r touch me gentl...,"[joy, fear, sadness]","[happy, anxious, sad]"
2,ABBA,As Good As New,ill never know why i had to go \r why i had t...,"[sadness, surprise, joy]","[sad, excited, happy]"
3,ABBA,Bang,making somebody happy is a question of give an...,"[joy, surprise, sadness]","[happy, excited, sad]"
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...,"[joy, surprise, sadness]","[happy, excited, sad]"
...,...,...,...,...,...
57645,Ziggy Marley,Good Old Days,irie days come on play \r let the angels fly ...,"[sadness, joy, surprise]","[sad, happy, excited]"
57646,Ziggy Marley,Hand To Mouth,power to the workers \r more power \r power ...,"[sadness, surprise, anger]","[sad, excited, angry]"
57647,Zwan,Come With Me,all you need \r is something ill believe \r ...,"[sadness, surprise, joy]","[sad, excited, happy]"
57648,Zwan,Desire,northern star \r am i frightened \r where ca...,"[sadness, fear, joy]","[sad, anxious, happy]"


In [25]:
import re
import nltk
nltk.download('stopwords')





[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer  #for stemmming
ps = PorterStemmer()

In [27]:
corpus = []

for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df['clean_lyrics'][i])

    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500,binary = True , ngram_range=(1,3))  #it give only 0 and 1  #2500 is for 2500 importent fitures
x = cv.fit_transform(corpus).toarray()

In [33]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
x[1]

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
x.shape

(57650, 2500)

In [37]:
y = pd.get_dummies(df['clean_lyrics'])
y = y.iloc[:, 1].values

In [38]:
y

array([False, False, False, ..., False, False, False])

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [47]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [48]:
y_pred=spam_detect_model.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score,classification_report

In [60]:
# score=accuracy_score(y_test,y_pred)
# print(score)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9998761 0.9998761 1.        0.9998761 0.9998761]
Average CV Score:  0.9999008796927271
Number of CV Scores used in Average:  5


In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

In [52]:
clf = DecisionTreeClassifier(random_state=42)

In [55]:
k_folds = KFold(n_splits = 5)

scores = cross_val_score(clf, X_train, y_train, cv = k_folds)

In [56]:
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9998761 0.9998761 1.        0.9998761 0.9998761]
Average CV Score:  0.9999008796927271
Number of CV Scores used in Average:  5


In [58]:
# score=accuracy_score(y_test,y_pred)
# print(score)